In [3]:
import pandas as pd
import numpy as np


In [4]:
df=pd.read_csv('sushi.csv')
df.head()

,Name,Rating,Reviews
0,Yama Sushi Sake Attitude,5.0,214
1,Uzumaki,4.5,669
2,Izakaya Osen - Los Angeles,4.5,1631
3,Roll Call - Koreatown,4.5,532
4,Shikibu Restaurant,4.5,409
